In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df1 = pd.read_csv('/kaggle/input/7k-books-with-metadata/books.csv')

In [ ]:
df1.head(10)

In [ ]:
df1.isnull().sum()

In [ ]:
df2 = df1.drop(columns=['subtitle'])

In [ ]:
df2.head()

In [ ]:
df2 = df2.dropna(subset=["description","categories","authors"])
df2.shape

In [ ]:
df2.isnull().sum()

In [ ]:
df2 = df2.drop(columns=['isbn13','isbn10','thumbnail','published_year','average_rating','num_pages','ratings_count'])

In [ ]:
df2 = df2.reset_index(drop=True)
df2.head()


In [ ]:
df2['categories'] = df2['categories'].apply(lambda x:x.split())
df2['description'] = df2['description'].apply(lambda x:x.split())

In [ ]:
df2['authors'][0]


In [ ]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    elif isinstance(x, str):
        return str.lower(x.replace(" ", ""))
    else:
        return ''

In [ ]:
features = ['authors', 'description', 'categories']
for feature in features:
    df2[feature] = df2[feature].apply(clean_data)

In [ ]:
df2.head()

In [ ]:
def merge_book_data(x):
    return ' '.join(x['description']) + ' ' + ' '.join(x['categories']) + ' ' + x['authors']

In [ ]:
df2[['tags']] = df2.apply(merge_book_data, axis=1)

In [ ]:
df2['tags'][3]

In [ ]:
df2.head()

In [ ]:
new = df2.drop(columns=['authors','description','categories'])

In [ ]:
new.shape

In [ ]:
x = new['tags'].values.tolist()
y = ''
for i in x:
    y = y + i
    
len(y)

In [ ]:
len(set(y.split()))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=10000,stop_words='english')

In [ ]:
vector = cv.fit_transform(new['tags']).toarray()

In [ ]:
vector.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vector)

In [ ]:
similarity.shape

In [ ]:
def recommend_content_based(book):
    index = new[new['title'] == book].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)

In [ ]:
 recommend_content_based("The Science of Harry Potter")

In [ ]:
 recommend_content_based("The Jane Austen Cookbook")

In [ ]:
 recommend_content_based("Wuthering Heights")

In [ ]:
 recommend_content_based("Love and Saint Augustine")

In [ ]:
recommend_content_based("Three Philosophies of Life")

In [ ]:
recommend_content_based("Macbeth")

In [ ]:
recommend_content_based("The Chronicles of Narnia (adult)")

In [ ]:
recommend_content_based("C++ Programmer's Notebook")

In [ ]:
new.sample(10)